In [130]:
#import libaries
import numpy as np
from numpy import random
import math

#ignore warnings about almost ill-conditioned matrices
import warnings
warnings.filterwarnings('ignore') 

In [131]:
#Aux methods:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def check_all_matrices(x):
  b = True
  for m in x:
    b = b and is_pos_def(m)
    if not b: return False
  return b

In [132]:
#generate dataset
n = 100 #datapoints
d = 3 #dimensions
c = 10 #number of clusters

def create_cov_matrices(cov_list):
  for m in cov_list:
    np.fill_diagonal(m, abs(random.normal(size=(d), scale=spread_of_cluster_centre)))

def generate_dataset(mean_list, cov_list, n):
  x_features = []
  y_labels = random.choice(c, n)
  for i in range(n):
    rand_c = y_labels[i]
    mean = mean_list[rand_c]
    cov = cov_list[rand_c]
    datapoint = np.random.multivariate_normal(mean,cov,(1))
    [datapoint] = datapoint
    x_features.append(datapoint)
  x_features = np.array(x_features)
  return (x_features, y_labels)

spread_of_cluster_centre = math.pi * 3
spread_around_clusters = math.pi * 3
cluster_centres = random.normal(size=(c,d), scale=spread_of_cluster_centre, loc=random.normal(spread_around_clusters))
print(cluster_centres)

temp = []
for c_i in range(c):
  temp.append(np.zeros((d,d), float))
cov_matrices_list = np.array(temp)
# z = np.zeros((d,d), float)
# cov_matrices_list = np.repeat(z[:, :, np.newaxis], c, axis=2)
# print(cov_matrices_list)
create_cov_matrices(cov_matrices_list)

# print(check_all_matrices(cov_matrices_list))
x_features, y_labels = generate_dataset(cluster_centres, cov_matrices_list, n)

# print(cluster_centres)
# print()
# print(cov_matrices_list)
# print()
# print(x_features)
# print()
# print(y_labels)



[[ 7.03759781 12.26858741 17.78002088]
 [13.15635891 13.71212604 18.27365551]
 [25.26178764 18.88502428  4.10873306]
 [30.8925023  12.07991752  2.38026208]
 [26.50494674 12.56332633 18.0363759 ]
 [13.470723    7.56398519 12.28781495]
 [17.14014492 10.73750889 -2.55976731]
 [-4.68607654  5.95847455 19.12726913]
 [17.49920746  0.33686929  4.2260267 ]
 [ 9.28390477 17.77847638 13.57408896]]


In [133]:
#Generate test set
x_features_test, y_labels_test = generate_dataset(cluster_centres, cov_matrices_list, n)

In [134]:
#OvA classifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

ova_classifier = OneVsRestClassifier(SVC()).fit(x_features, y_labels)
print(ova_classifier.score(x_features_test, y_labels_test))
print(y_labels_test == ova_classifier.predict(x_features_test))

0.86
[ True  True  True  True  True False  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True False  True False False  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True False  True
  True  True False  True  True  True  True False  True  True  True False
  True  True  True  True]


In [135]:
#OvO classifier
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
X_train, X_test, y_train, y_test = train_test_split(
    x_features, y_labels, test_size=0.33, shuffle=True, random_state=42)
ovo_classifier = OneVsOneClassifier(
    LinearSVC(random_state=42)).fit(X_train, y_train)
print(ovo_classifier.score(x_features_test, y_labels_test))
print(y_labels_test == ovo_classifier.predict(x_features_test))

0.81
[ True  True  True  True  True  True  True  True  True  True False  True
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True False  True  True False  True False  True  True  True  True  True
  True  True  True  True False  True  True False  True  True False  True
  True False False  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True False  True  True  True False  True
  True  True False  True  True False  True  True  True  True  True False
  True  True False  True]
